<a href="https://colab.research.google.com/github/dave-does-data/llm-qa-rlhf/blob/main/llm_qa_plhf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!git clone https://github.com/dave-does-data/llm-qa-rlhf.git
!pip install -r llm-qa-rlhf/requirements.txt

In [ ]:
!wandb login

In [ ]:
!python llm-qa-rlhf/src/trl/sft_trainer.py \
    --model_name meta-llama/Llama-2-7b-hf \
    --dataset_name dave-does-data/databricks-dolly-qa-subset-7k \
    --log_with 'wandb' \
    --load_in_4bit \
    --use_peft \
    --batch_size 4 \
    --gradient_accumulation_steps 2